# Train

## Library Import

In [1]:
PROJECT = "RFCX"
EXP_NUM = "0"
EXP_TITLE = "Baseline"
EXP_NAME = "exp_" + EXP_NUM + "_" + EXP_TITLE
IS_WRITRE_LOG = True
print('expname:' + EXP_NAME)

expname:exp_0_Baseline


In [2]:
# library import
import numpy as np
import pandas as pd
import os
import tqdm
import random
import time
import math
import csv
import torch
import torch.nn as nn
from time import sleep
from torch.nn import functional as F
from torch.optim import Adam, AdamW
from torchvision.models import resnet18, resnet34, resnet50
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import Subset
import torch.utils.data as torchdata
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from contextlib import contextmanager
from typing import Optional
import logging
from numpy.random import beta
from pathlib import Path

from fastprogress.fastprogress import master_bar, progress_bar

from torchviz import make_dot


from conformer import ConformerConvModule
from conformer import ConformerBlock

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

from torch.utils.tensorboard import SummaryWriter

import wandb


pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)



## Configuration

In [3]:
# use GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [4]:
# sample data
sample = torch.from_numpy(np.load('melspec.npy'))
print(sample.shape)
print(sample[np.newaxis, :, :].shape)
# channel = sample.unsqueeze(0)
# batch = channel.unsqueeze(0)
# print(batch.shape)

torch.Size([2813, 128])
torch.Size([1, 2813, 128])


In [5]:
params = {
    "SEED":               42,
    "INPUT":              Path("../input/rfcx-species-audio-detection/train"),
    "TRAIN_AUDIO_ROOT":   Path("../input/rfcx-species-audio-detection/train_mel"),
    "TEST_AUDIO_ROOT":    Path("../input/rfcx-species-audio-detection/test_mel"),
    "TRAIN_TP":           Path("../input/rfcx-species-audio-detection/train_tp.csv"),
    "SUB":                Path("../input/rfcx-species-audio-detection/sample_submission.csv"),
    "DIM":                sample.shape[1],
    "SEQ_LEN":            sample.shape[0],
    "KERNEL_SIZE":        3,
    "POOL_SIZE":          2,
    "POOL_STRIDE":        2,
    "NUM_BIRDS":          24,
    "N_FOLDS":            5,
    "BATCH_NUM":          40,
    "VALID_BATCH_NUM":    40,
    "EPOCH_NUM":          50,
    "DROPOUT":            0.35,
    "lr":                 0.0001,
    "eta_min":            1e-5,
    "t_max":              10,
}
wandb.init(config=params, project=PROJECT)
config = wandb.config
wandb.run.name = EXP_NAME
wandb.run.save()
print(config.SEED)

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: tatsuya-takahashi (use `wandb login --relogin` to force relogin)


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.
42


In [6]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
set_seed(config.SEED)

## Audio Transform

In [7]:
# transform all audio to 2d
# run audio_transformer

## Preprocessing

In [8]:
# Spec Augment








In [9]:
# any other augment here...









## Modeling

In [10]:
# Conformer
# https://arxiv.org/abs/2005.08100
class RainforestTransformer(nn.Module):
    def __init__(self):
        super(RainforestTransformer, self).__init__()         

        self.conv = nn.Conv2d(1, 1, config.KERNEL_SIZE)
        self.linear = nn.Linear(int(
                                    (
                                        ((config.DIM - config.KERNEL_SIZE + 1) - config.POOL_SIZE) / config.POOL_STRIDE
                                    ) + 1
                                ), config.DIM)
        self.dropout = nn.Dropout(config.DROPOUT)
        
        self.conformerblock = ConformerBlock(
            dim = config.DIM,
            dim_head = 64,
            heads = 8,
            ff_mult = 4,
            conv_expansion_factor = 2,
            conv_kernel_size = 31,
            attn_dropout = config.DROPOUT,
            ff_dropout = config.DROPOUT,
            conv_dropout = config.DROPOUT
        )
        self.decoder = nn.Linear(1 * int((((config.SEQ_LEN - config.KERNEL_SIZE + 1) -  config.POOL_SIZE) / config.POOL_STRIDE) + 1) * config.DIM, config.NUM_BIRDS)

        # devided by stride
        
    def forward(self, x):
        h = F.relu(self.conv(x))
        h = F.max_pool2d(h, config.POOL_SIZE, stride=config.POOL_STRIDE)
        h = self.linear(h)
        h = h.transpose(0, 1)[0] # transpose batch and channel to delet channel dimension
        h = self.conformerblock(h)
        h = h.view(-1, 1 * int((((config.SEQ_LEN - config.KERNEL_SIZE + 1) -  config.POOL_SIZE) / config.POOL_STRIDE) + 1) * config.DIM)
        out = self.decoder(h)
        return out

## Metric

In [11]:
# from official code https://colab.research.google.com/drive/1AgPdhSp7ttY18O3fEoHOQKlt_3HJDLi8#scrollTo=cRCaCIb9oguU
def _one_sample_positive_class_precisions(scores, truth):
    """Calculate precisions for each true class for a single sample.

    Args:
      scores: np.array of (num_classes,) giving the individual classifier scores.
      truth: np.array of (num_classes,) bools indicating which classes are true.

    Returns:
      pos_class_indices: np.array of indices of the true classes for this sample.
      pos_class_precisions: np.array of precisions corresponding to each of those
        classes.
    """
    num_classes = scores.shape[0]
    pos_class_indices = np.flatnonzero(truth > 0)
    # Only calculate precisions if there are some true classes.
    if not len(pos_class_indices):
        return pos_class_indices, np.zeros(0)
    # Retrieval list of classes for this sample.
    retrieved_classes = np.argsort(scores)[::-1]
    # class_rankings[top_scoring_class_index] == 0 etc.
    class_rankings = np.zeros(num_classes, dtype=np.int)
    class_rankings[retrieved_classes] = range(num_classes)
    # Which of these is a true label?
    retrieved_class_true = np.zeros(num_classes, dtype=np.bool)
    retrieved_class_true[class_rankings[pos_class_indices]] = True
    # Num hits for every truncated retrieval list.
    retrieved_cumulative_hits = np.cumsum(retrieved_class_true)
    # Precision of retrieval list truncated at each hit, in order of pos_labels.
    precision_at_hits = (
            retrieved_cumulative_hits[class_rankings[pos_class_indices]] /
            (1 + class_rankings[pos_class_indices].astype(np.float)))
    return pos_class_indices, precision_at_hits


def calculate_per_class_lwlrap(truth, scores):
    """Calculate label-weighted label-ranking average precision.

    Arguments:
      truth: np.array of (num_samples, num_classes) giving boolean ground-truth
        of presence of that class in that sample.
      scores: np.array of (num_samples, num_classes) giving the classifier-under-
        test's real-valued score for each class for each sample.

    Returns:
      per_class_lwlrap: np.array of (num_classes,) giving the lwlrap for each
        class.
      weight_per_class: np.array of (num_classes,) giving the prior of each
        class within the truth labels.  Then the overall unbalanced lwlrap is
        simply np.sum(per_class_lwlrap * weight_per_class)
    """
    assert truth.shape == scores.shape
    num_samples, num_classes = scores.shape
    # Space to store a distinct precision value for each class on each sample.
    # Only the classes that are true for each sample will be filled in.
    precisions_for_samples_by_classes = np.zeros((num_samples, num_classes))
    for sample_num in range(num_samples):
        pos_class_indices, precision_at_hits = (
            _one_sample_positive_class_precisions(scores[sample_num, :],
                                                  truth[sample_num, :]))
        precisions_for_samples_by_classes[sample_num, pos_class_indices] = (
            precision_at_hits)
    labels_per_class = np.sum(truth > 0, axis=0)
    weight_per_class = labels_per_class / float(np.sum(labels_per_class))
    # Form average of each column, i.e. all the precisions assigned to labels in
    # a particular class.
    per_class_lwlrap = (np.sum(precisions_for_samples_by_classes, axis=0) /
                        np.maximum(1, labels_per_class))
    # overall_lwlrap = simple average of all the actual per-class, per-sample precisions
    #                = np.sum(precisions_for_samples_by_classes) / np.sum(precisions_for_samples_by_classes > 0)
    #           also = weighted mean of per-class lwlraps, weighted by class label prior across samples
    #                = np.sum(per_class_lwlrap * weight_per_class)
    return per_class_lwlrap, weight_per_class

## Transforms

In [12]:
# transforms
train_transform = transforms.Compose([
    # transforms.RandomCrop((128, 313), pad_if_needed=True, padding_mode="constant"),
    # transforms.ToTensor()
])
valid_transform = transforms.Compose([
    # transforms.CenterCrop((128, 313)),
    # transforms.ToTensor()
])
label_transform = transforms.Compose([
    # transforms.ToTensor()
])

## Dataset

In [13]:
# Data load
df_train_tp = pd.read_csv(config.TRAIN_TP)

# add column
for col in range(24):
    df_train_tp[col] = 0

# bit
for index, row in df_train_tp.iterrows():
    specId = row["species_id"]
    for col in range(24):
        if int(specId) == col:
            df_train_tp.iloc[index, df_train_tp.columns.get_loc(col)] = 1

# grouping
df_train_tp = df_train_tp.groupby("recording_id", as_index=False).max()

# check
df_train_tp[df_train_tp["recording_id"] == "77299bde7"].head(100)


,recording_id,species_id,songtype_id,t_min,f_min,t_max,f_max,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
538,77299bde7,21,1,42.3787,3750.0,43.472,5531.25,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [14]:
# load data
record_ids = []
labels = []
for index, row in df_train_tp.iterrows():
    record_ids.append(row.values[0])
    labels.append(row.values[7:31])

labels = np.array(labels).astype(float)

print(record_ids[0])
print(labels[0])


003bec244
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [15]:
class RainforestDatasets(torch.utils.data.Dataset):
    def __init__(self, transform = None, train = True):
        self.transform = transform
        self.train = train

        # data load
        self.labelset = labels
        self.dataset = []
        for rid in record_ids:
            # read npy
            melspec = np.load(os.path.join(config.TRAIN_AUDIO_ROOT, rid + ".npy"))
            # melspec = torch.from_numpy(melspec)
            # melspec = melspec.unsqueeze(0) # add channel for first convolution
            melspec = melspec[np.newaxis, :, :]
            self.dataset.append(melspec)

        self.dataset = np.array(self.dataset).astype(float)
        self.datanum = len(self.dataset)
        

    def __len__(self):
        return self.datanum

    def __getitem__(self, idx):
        # get data
        out_label = self.labelset[idx]
        out_data = self.dataset[idx]

        # print(type(out_label))
        
        # transform label
        # out_data = self.transform(out_data)
        # out_label = label_transform(out_label)

        # print(type(out_data.shape))
        # print(out_label)

        # print(out_data.shape)

        out_label = torch.from_numpy(out_label).float()
        out_data = torch.from_numpy(out_data).float()
        # print(type(out_data))
        # print(type(np.array(out_label)))
        # print(out_data.shape)

        return out_data, out_label

## Train

In [16]:
# model = RainforestTransformer()
# make_dot(model(), params=dict(model.named_parameters()))

In [17]:
def train():

    # Stratified k-fold
    # skf = StratifiedKFold(n_splits=config.N_FOLDS, shuffle=True, random_state=config.SEED)
    msss = MultilabelStratifiedShuffleSplit(n_splits=config.N_FOLDS, test_size=0.2, random_state=config.SEED)

    # Read dataset
    train_datasets = RainforestDatasets(transform=train_transform)
    valid_datasets = RainforestDatasets(transform=valid_transform)

    best_epochs = []
    best_lwlraps = []
    best_lwlrap = 0.
    best_epoch = 0

    if IS_WRITRE_LOG:
        writer = SummaryWriter(log_dir="./logs/" + EXP_NAME)

    # k-fold
    for kfoldidx, (train_index, valid_index) in enumerate(msss.split(labels, labels)):

        # model 
        model = RainforestTransformer()
        model.to(device)

        if IS_WRITRE_LOG:
            wandb.watch(model)

        # criterion
        criterion = nn.BCEWithLogitsLoss().cuda()

        # optimizer
        optimizer = Adam(params=model.parameters(), lr=config.lr, amsgrad=False)
        # print(optimizer)

        # scheduler
        scheduler = CosineAnnealingLR(optimizer, T_max=config.t_max, eta_min=config.eta_min)

        # train
        train_subset = Subset(train_datasets, train_index)
        train_loader = DataLoader(train_subset, batch_size=config.BATCH_NUM, shuffle=True)

        # validation
        valid_subset = Subset(valid_datasets, valid_index)
        valid_loader = DataLoader(valid_subset, batch_size=config.VALID_BATCH_NUM, shuffle=False)

        # epoch
        mb = master_bar(range(config.EPOCH_NUM))
        mb.names = ['avg_loss', 'avg_val_loss', 'lwlrap']

        # Epoch
        for epoch in mb:

            # start time
            start_time = time.time()

            # train mode
            model.train()

            # init loss
            avg_loss = 0.

            # batch training
            train_batch_preds = []
            train_batch_labels = []
            for x_batch, y_batch in progress_bar(train_loader, parent=mb):
                preds = model(x_batch.to(device))
                loss = criterion(preds, y_batch.to(device)) # It dosen't need Sigmoid, because BCE includes sigmoid function.

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                avg_loss += loss.item() / len(train_loader)

                # add preds
                train_batch_preds.extend(preds.detach().cpu().numpy())
                train_batch_labels.extend(np.array(y_batch))

            # calc score
            score, weight = calculate_per_class_lwlrap(np.array(train_batch_labels), np.array(train_batch_preds))
            train_lwlrap = (score * weight).sum()

            # validation mode
            model.eval()
            valid_preds = np.zeros((len(valid_index), config.NUM_BIRDS))
            avg_val_loss = 0.

            # validation
            for i, (x_batch, y_batch) in enumerate(valid_loader):
                preds = model(x_batch.to(device)).detach()
                loss = criterion(preds, y_batch.to(device))

                preds = torch.sigmoid(preds)
                valid_preds[i * config.VALID_BATCH_NUM: (i+1) * config.VALID_BATCH_NUM] = preds.cpu().numpy()

                avg_val_loss += loss.item() / len(valid_loader)

            # calc score
            score, weight = calculate_per_class_lwlrap(labels[valid_index], valid_preds)
            lwlrap = (score * weight).sum()

            # update lr
            scheduler.step()

            # tensorboard
            if IS_WRITRE_LOG:
                # tensorboard
                writer.add_scalar("train_loss/fold-" + str(kfoldidx), avg_loss, epoch)
                writer.add_scalar("valid_loss/fold-" + str(kfoldidx), avg_val_loss, epoch)
                writer.add_scalar("train_lwlrap/fold-" + str(kfoldidx), train_lwlrap, epoch)
                writer.add_scalar("valid_lwlrap/fold-" + str(kfoldidx), lwlrap, epoch)

                # wandb
                wandb.log({
                    f'loss/train/fold-{kfoldidx}': avg_loss,
                    f'lwlrap/train/fold-{kfoldidx}': train_lwlrap,
                    f'loss/validation/fold-{kfoldidx}': avg_val_loss,
                    f'lwlrap/validation/fold-{kfoldidx}': lwlrap,
                    'epoch': epoch
                })


            if (epoch + 1) % 5 == 0:
                elapsed = time.time() - start_time
                mb.write(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  val_lwlrap: {lwlrap:.6f}  time: {elapsed:.0f}s')
        
            if lwlrap > best_lwlrap:
                best_epoch = epoch + 1
                best_lwlrap = lwlrap
                torch.save(model.state_dict(), 'weight_best_fold' + str(kfoldidx) +'.pt')
            
        best_epochs.append(best_epoch)
        best_lwlraps.append(best_lwlrap)

    if IS_WRITRE_LOG:
        writer.close()
    
    return {
        'best_epoch': best_epochs,
        'best_lwlrap': best_lwlraps,
    }




In [18]:
result = train()
print(result)

{'best_epoch': [33, 33, 33, 33, 33], 'best_lwlrap': [0.5826794957184678, 0.5826794957184678, 0.5826794957184678, 0.5826794957184678, 0.5826794957184678]}


## Predict

In [19]:
# prediction
models = []
for fold in range(config.N_FOLDS):
    # load network
    model = RainforestTransformer()
    model.load_state_dict(torch.load('weight_best_fold' + str(fold) +'.pt'))
    model.to(device)
    model.eval()
    models.append(model)


In [20]:
# write submission
with open('submission.csv', 'w', newline='') as csvfile:
    submission_writer = csv.writer(csvfile, delimiter=',')
    submission_writer.writerow(['recording_id','s0','s1','s2','s3','s4','s5','s6','s7','s8','s9','s10','s11',
                               's12','s13','s14','s15','s16','s17','s18','s19','s20','s21','s22','s23'])
    
    test_files = os.listdir(config.TEST_AUDIO_ROOT)
    print(len(test_files))
    
    # Every test file is split on several chunks and prediction is made for each chunk
    for i in range(0, len(test_files)):
        # read data
        X_test = torch.from_numpy(np.load(os.path.join(config.TEST_AUDIO_ROOT, test_files[i])))
        X_test = X_test.unsqueeze(0) # add channel for first convolution
        X_test = X_test.unsqueeze(0) # add batch for network
        X_test = X_test.to(device)

        # predict
        output_list = []
        for m in models:
            output = m(X_test)
            maxed_output = torch.max(output, dim=0)[0]
            maxed_output = maxed_output.cpu().detach()
            output_list.append(maxed_output)
        avg_maxed_output = torch.mean(torch.stack(output_list), dim=0)
        
        file_id = str.split(test_files[i], '.')[0]
        write_array = [file_id]
        
#         for out in maxed_output:
        for out in avg_maxed_output:
            write_array.append(out.item())
    
        submission_writer.writerow(write_array)
        
        if i % 100 == 0 and i > 0:
            print('Predicted for ' + str(i) + ' of ' + str(len(test_files) + 1) + ' files')

1992
Predicted for 100 of 1993 files
Predicted for 200 of 1993 files
Predicted for 300 of 1993 files
Predicted for 400 of 1993 files
Predicted for 500 of 1993 files
Predicted for 600 of 1993 files
Predicted for 700 of 1993 files
Predicted for 800 of 1993 files
Predicted for 900 of 1993 files
Predicted for 1000 of 1993 files
Predicted for 1100 of 1993 files
Predicted for 1200 of 1993 files
Predicted for 1300 of 1993 files
Predicted for 1400 of 1993 files
Predicted for 1500 of 1993 files
Predicted for 1600 of 1993 files
Predicted for 1700 of 1993 files
Predicted for 1800 of 1993 files
Predicted for 1900 of 1993 files
